<a href="https://colab.research.google.com/github/PondKann/KKU_data_mining/blob/master/Data_Retail2_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. นำเข้าข้อมูล และ Package ที่ต้องการ

In [1]:
import pandas as pd
import os 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/DataViz_2021'
os.listdir(path)

['pm-18-jan-2021.csv',
 'covid-19-daily_data_dictionary.xlsx',
 'income.csv',
 'income2.xls',
 'จัดซื้อ 57-58.xls',
 'online_retail_II.xlsx',
 'online_retail_II.csv']

In [ ]:
income2_file_path = os.path.join(path,'income2.xls')
print(income2_file_path)
income3= pd.read_excel(income2_file_path,sheet_name=0,skiprows=3,header=0)
data_income = income3.iloc[2:-4]

In [19]:
data_path = os.path.join(path,'online_retail_II.xlsx')
print(data_path)

/content/drive/My Drive/DataViz_2021/online_retail_II.xlsx


In [114]:
df1= pd.read_excel(data_path,sheet_name=0,header=0)

In [22]:
df2= pd.read_excel(data_path,sheet_name=1,header=0)

In [115]:
df1

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [116]:
df1[df1['InvoiceDate']=='1/4/2010  9:24:00 AM']

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
45228,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.5,12346.0,United Kingdom


In [ ]:
df1[df1['InvoiceDate']=='12/9/2010  8:01:00 PM']

ตัดปี2010 ของชีทแรกออก

In [ ]:
df1 = df1.iloc[45228:,:525460]

In [118]:
df1.shape

(480233, 8)

In [42]:
df2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df1.info()

In [119]:
print(f'ขนาดของ df1 คือ {df1.shape} ')
print(f'ขนาดของ df2 คือ {df2.shape} ')

ขนาดของ df1 คือ (480233, 8) 
ขนาดของ df2 คือ (541910, 8) 


In [120]:
dataAll = pd.concat([df1,df2])
dataAll

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
45228,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0,United Kingdom
45229,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0,United Kingdom
45230,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0,United Kingdom
45231,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN,United Kingdom
45232,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


ตัดข้อมูลที่เป็น POST ออกจากตารางใหญ่

In [121]:
dataAll_noPOST = dataAll[dataAll['StockCode']!='POST']
dataAll_noPOST.shape

(1020074, 8)

In [122]:
dataAll[dataAll['StockCode']=='POST'].shape[0]

2069

In [123]:
print(f'ขนาดของ All คือ {dataAll.shape[0]} ')
print(f'ขนาดของ ลบ post คือ {dataAll_noPOST.shape[0]} ')
print(f'ขนาดที่หายไป คือ {(dataAll.shape[0] )-  (dataAll_noPOST.shape[0]) } ')

ขนาดของ All คือ 1022143 
ขนาดของ ลบ post คือ 1020074 
ขนาดที่หายไป คือ 2069 


ตัดข้อมูลที่ถูกยกเลิกออก ให้อยู่ในตาราง dataAll_noC

In [ ]:
dataAll_noPOST['Description'] = dataAll_noPOST['Description'].str.strip()      
dataAll_noPOST.dropna(axis=0, subset=['Invoice'], inplace=True)
dataAll_noPOST['Invoice'] = dataAll_noPOST['Invoice'].astype('str')             
dataAll_noC = dataAll_noPOST[~dataAll_noPOST['Invoice'].str.contains('C')] 

In [125]:
print(f'ขนาดของ ลบ post คือ {dataAll_noPOST.shape[0]} ')
print(f'ขนาดของ ลบ cancel คือ {dataAll_noC.shape[0]} ')
print(f'ขนาดที่หายไป คือ {(dataAll_noPOST.shape[0] ) -   (dataAll_noC.shape[0]) } ')

ขนาดของ ลบ post คือ 1020074 
ขนาดของ ลบ cancel คือ 1001813 
ขนาดที่หายไป คือ 18261 


In [126]:
dataAll_noC

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
45228,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0,United Kingdom
45230,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0,United Kingdom
45231,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN,United Kingdom
45232,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN,United Kingdom
45233,493413,21723,ALPHABET HEARTS STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [127]:
dataAll_noC.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [128]:
dataAll_noC.shape

(1001813, 8)

In [129]:
dataAll_noC.to_csv('dataAll.csv')